In [41]:
#Set up imports
from bs4 import BeautifulSoup, NavigableString, Tag, SoupStrainer
from pprint import pprint
from urllib.parse import urljoin
import time
import base64

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import tkinter as tk
import os
import sys
from nltk.tokenize import word_tokenize, sent_tokenize
from num2words import num2words
from pathlib import Path
from google.protobuf.json_format import ParseDict
from Pro7_File_API_Python import presentation_pb2  # Used to decode *.pro files
import re
from google.protobuf.message import Message as ProtobufMessageType
import proto

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = Path.expanduser(Path.home())


In [42]:
#Define functions
def create_cue_index_dict(pro7_file_obj,element_number=1,suppress_output = True):
    if element_number == None:
        el = 1
    else:
        el = element_number
    
    slide_order_list_raw = []
    slide_order_list = []
    cue_list = []
    #cue_UUID_list = []
    cue_dict = {}
    cue_index_dict = {}

    slide_order_list_raw = pro7_file_obj.cue_groups[0].cue_identifiers

    for sl in slide_order_list_raw:
        UUID=str(sl).split("\"")[1]
        if suppress_output == False:
            print(UUID,end="\n")
        slide_order_list.append(UUID)
        
    #print(len(slide_order_list),end="\n")


    cue_list = pro7_file_obj.cues

    i=0
    for c in cue_list:
        try:
            #escapes = ''.join([chr(char) for char in range(1, 32)])
            #translator = str.maketrans('', '', escapes)
            cue_UUID = str(c.uuid).split("\"")[1]
            #tab_stops = c.actions[0].slide.presentation.base_slide.elements[el].element.text.attributes.paragraph_style.tab_stops
            #text_element = c.actions[0].slide.presentation.base_slide.elements[el].element.text.rtf_data.decode(encoding='cp1252')
            #text_line = text_element.split("0\n")[1].split("\n")[1].translate(translator)[:-1]
            #cue_dict[cue_UUID]=text_line
            cue_index_dict[slide_order_list.index(cue_UUID)] = i
            i+=1

        except IndexError:
            print("Index Error encountered in generating cue index dictionary")
            i+=1

    #ordered_cue_list = []
    #print("Reconstructed slide text:")
    #for UUID in slide_order_list:
    #    print(cue_dict[UUID])

    cue_index_dict = dict(sorted(cue_index_dict.items()))

    return cue_index_dict
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# This regex matches characters preceded by start of line or an underscore.
_RE_FIND_CHARS_TO_UPPERCASE = re.compile(r"(?:_|^)([a-z])")


def convert_protobuf_to_proto_plus(message):
    """Converts a protobuf message to a proto-plus message.

    Args:
        message: an instance of google.protobuf.message.Message

    Returns:
        A proto_plus version of the protobuf proto.
    """
    if isinstance(message, ProtobufMessageType):
        return proto.Message.wrap(message)
    elif isinstance(message, proto.Message):
        return message
    else:
        raise TypeError(
            f"Cannot convert type {type(message)} to a proto_plus protobuf."
        )
def get_source_info(header_text):
    reading_source = {}
    book_and_number = ""
    type_header = ""
    type = ""
    preface = ""
    type_header = header_text.split(":")[0].split(" ")[-1].strip()
    type_title = header_text.split(":")[0].replace(" ","-").strip()
    if type_header == "Reading":
        type = "reading"
    elif type_header == "Psalm":
        type = "psalm"
    elif type_header == "Gospel":
        type = "gospel"
    elif type_header == "Prayer":
        type = "prayer"

    if type == "prayer":
        preface = header_text.split(":")[1].strip()
        reading_source["type"] = type
        reading_source["preface"] = preface
        reading_source["book"] = ""
        reading_source["number"] = ""
        reading_source["title"] = "Prayer-of-the-day" #type_title
        return reading_source
    else:
        book_and_number = header_text.split(":")[1].strip()
        number = book_and_number.split(" ")[-1]
        book = " ".join(book_and_number.split()[:-1])
        reading_source["type"] = type
        reading_source["book"] = book
        reading_source["number"] = number
        reading_source["title"] = type_title

        return reading_source
# Write to file (add new line at end and catch errors)
def write_file_line(file, text):
    try:
        file.write(text + "\n")
        # flush log data to disk immediately
        file.flush()
    except BaseException as err:
        # Something went wrong writing the log file. No further logging will be captured! Display error to user.
        tk.messagebox.showinfo(title="Error!", message=repr(err))
def open_reading_template(filename):
    #filename = "Reading_template_slides"

    filename_in = "./"+filename+".pro"

    open_file = filename_in
    if open_file != "":  # if folder dialog was not cancelled
        open_file = Path(open_file)
        print("Template loaded: " + str(open_file), end = ". ")

        if str(open_file).split(".")[-1] == "pro":
            print("Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...")
            pro7_file_obj = presentation_pb2.Presentation()

            file1 = open(open_file, mode='rb')

            parse_error = False
            try:
                pro7_file_obj.ParseFromString(file1.read())
                
            except BaseException as err:
                #write_file_line(log_file, 'ERROR: ' + repr(err) + ' occurred trying to parse ' + file1.name)
                parse_error = True
            file1.close()
    return pro7_file_obj
def tokenize_reading(a):
    a_lines = []
    a_sents = []
    a_clean = a.replace('“','"').replace('”','"')
    
    
    a_split = a_clean.split('"')

    
    a_lines_withblanks = []
    for i in range(len(a_split)):    
        if i%2 == 0:
            a_lines_withblanks.append(a_split[i].strip())
        else:
            a_lines_withblanks.append('"'+a_split[i].strip()+'"')

    a_lines = [line for line in a_lines_withblanks if line != ""]


    #Insert a_lines into a_list at current position
    #a_list.insert(a_list.index(a),a_lines)

    for i in range(0,len(a_lines)):
        #Removing this, but may need to be reinstated and figured out further…
        #if len(a_lines)==1 or i<len(a_lines)-1: #TODO Figure this out and remove it
        ###print(a_lines[i])
        a_lines[i]=a_lines[i].lstrip()
        
        if a_lines[i][:-1][0]==",":
            a_sents.extend(re.split(r'([.?!])'),a_lines[i])
            
        elif a_lines[i][0]=="\"":
            if a_lines[i-1][-1:][0]==",":
                tmp_line = a_sents.pop()
                a_sents.append(tmp_line+" "+a_lines[i])
                
            else:
                a_sents.append(a_lines[i])
                
        else:
            a_sents.extend(sent_tokenize(a_lines[i]))
                
        #i+=1
    
    print("\n")
    return a_sents

def slides_from_reading(reading, char_limit, show=False): 
    slides = []
    sent_comb = []
    slide_n=0
    i = 0
    #TODO — Fix this to allow for slides with huge quotes to break into sentences based on the character count (See 4-9-2023 Gospel for an example)
    #Go through the list of sentences and assemble slides of proper character count
    while i in range(len(reading)):
        ###To assemble the next slide:

        #First, reset our character counter
        count = 0

        #Create a new list item to hold info about this slide by adding the next sentence to the list at the base level of the tree
        sent_comb.append([reading[i]])
        count += len(reading[i])

        #count_with_next = count + len(reading[i+1])
        #Now, check out character count against the limit and add a loop to monitor it as we assemble
        while (count < char_limit) and (i in range(len(reading)-1)):
            if count + len(reading[i+1]) > char_limit:
                break
            #We're in here because we have room on our slide.
            #So add the next sentence
            i+=1
            sent_comb[slide_n].append(reading[i])

            #And update our character counter
            count += len(reading[i])

        #If we're out of the loop, we've hit our character limit for this slide

        #So print out the info about this slide
        #print("Slide ",slide_n,";",sent_comb[slide_n],end="\n\n")

        #Update our slide counter to go to the next slide
        slide_n+=1

        #And update our sentence counter to go to the next sentence
        i+=1

    #Now join the sentences in each slide into a single string
    i=1
    for slide in sent_comb:
        slides.append(" ".join(slide))
        if show == True:
            print("Slide:",i,"Length:",len(slides[i-1]),":",slides[i-1])
        i+=1
    return slides

#Create a presentation dictionary object, then write our text lines out to the corresponding slides/elements
def make_reading_presentation(reading_slides,reading_source,suppress_output=True):

    if reading_source['type'] == "psalm-spoken":
        #Open psalm template file
        pro7_file_obj = open_reading_template("Psalm_template_spoken")
        preface_slides = 1
        cue_index_dict = create_cue_index_dict(pro7_file_obj,0)
        el = 0
    elif reading_source['type'] == "psalm-chanted":
        #Open psalm template file
        pro7_file_obj = open_reading_template("Psalm_template_chanted")
        preface_slides = 1
        cue_index_dict = create_cue_index_dict(pro7_file_obj,0)
        #Set location of primary substition text element
        el = 1
    elif reading_source['type'] == "prayer":
        #Open prayer template file
        pro7_file_obj = open_reading_template("Prayer_of_day_template")
        preface_slides = 1
        cue_index_dict = create_cue_index_dict(pro7_file_obj)
        el = 1
    else:
        #Open reading template file
        pro7_file_obj = open_reading_template("Reading_template_slides")
        preface_slides = 2
        cue_index_dict = create_cue_index_dict(pro7_file_obj)
        #Set location of primary substition text element
        el = 1


    #if reading_source['type'] != "prayer":
    if (suppress_output==False):
        print(cue_index_dict)

    #Convert protobuf to proto-plus
    proto_pres_obj = convert_protobuf_to_proto_plus(pro7_file_obj)
    proto_pres_obj_dict = proto.Message.to_dict(proto_pres_obj)


    for slide_number in range(0,preface_slides):
        n=cue_index_dict[slide_number] #+starting_offset]
       
        preface_text = ""
        number = ""
        if reading_source['type'] == "reading":
            preface_text = "A reading from"
            number = num2words(reading_source['number'],to='ordinal')

        elif reading_source['type'] == "gospel":
            preface_text = "The holy gospel according to"
            number = num2words(reading_source['number'],to='ordinal')

        elif (reading_source['type'] == "psalm-chanted") or (reading_source['type'] == "psalm-spoken"):
            preface_text = ""
            number = reading_source['number']
            

        elif reading_source['type'] == "prayer":
            preface_text = reading_source['preface']
            number = ""
        

        book = reading_source['book']
        text_line_64 = proto_pres_obj_dict['cues'][n]['actions'][0]['slide']['presentation']['base_slide']['elements'][0]['element']['text']['rtf_data'] #.decode(encoding='cp1252')
        text_line = base64.b64decode(text_line_64).decode(encoding='cp1252')
        text_line_mod = text_line.replace("@@@",number)
        text_line_mod = text_line_mod.replace("&&&",preface_text).replace("$$$",book)
        if reading_source['type'] == "prayer":
            text_line_mod = text_line_mod.replace("%%%",reading_slides[0])
        text_line_mod_reencode = base64.b64encode(text_line_mod.encode(encoding='cp1252'))
        #text_line_mod_reencode = base64.b64encode(text_line_mod.encode(encoding="utf-8"))
        proto_pres_obj_dict['cues'][n]['actions'][0]['slide']['presentation']['base_slide']['elements'][0]['element']['text']['rtf_data'] = text_line_mod_reencode

    for slide_number in range(preface_slides,len(reading_slides)+preface_slides,1):
        if (reading_source['type'] != "prayer"):
            n=cue_index_dict[slide_number] #+starting_offset]
            text_to_insert = reading_slides[slide_number-preface_slides]
            text_line_64 = proto_pres_obj_dict['cues'][n]['actions'][0]['slide']['presentation']['base_slide']['elements'][el]['element']['text']['rtf_data'] #.decode(encoding='cp1252')
            text_line = base64.b64decode(text_line_64).decode(encoding='cp1252')
            text_line_mod = text_line.replace("%%%"," "+text_to_insert).replace("\u2003"," ")
            text_line_mod_reencode = base64.b64encode(text_line_mod.encode(encoding='cp1252')) #"utf-8"))
            proto_pres_obj_dict['cues'][n]['actions'][0]['slide']['presentation']['base_slide']['elements'][el]['element']['text']['rtf_data'] = text_line_mod_reencode
    
    '''
    #Now that we've made all the slides we need for the reading, delete extraneous slides from the template
    uuids_to_delete = []
    for slide_number in range(len(reading_slides)+2,len(cue_index_dict),1):
        uuids_to_delete.append(slide_order_list[cue_index_dict[slide_number]])


    print("Deleting extra slides:") 
    for uuid in uuids_to_delete:
        for i in range(len(proto_pres_obj_dict['cues'])):
            if proto_pres_obj_dict['cues'][i]['uuid']['string'] == uuid:
                print("Deleting slide with UUID: "+uuid)
                del proto_pres_obj_dict['cues'][i]
                break
        for i in range(len(proto_pres_obj_dict['cue_groups'][0]['cue_identifiers'])):
            if proto_pres_obj_dict['cue_groups'][0]['cue_identifiers'][i]['string'] == uuid:
                print("Deleting slide reference with UUID: "+uuid)
                del proto_pres_obj_dict['cue_groups'][0]['cue_identifiers'][i]
                break


        #print(proto_pres_obj_dict['cues'][uuid])
        #del proto_pres_obj_dict['cues'][uuid]
    '''

    #When we're all done, use the ParseDict function to convert our dictionary back into a protobuf object using the Presentation class
    proto_pres_obj_mod = ParseDict(proto_pres_obj_dict,presentation_pb2.Presentation())

    #Write output file
    if reading_source['type'] == "psalm-spoken":
        filename_add = "-spoken"
    elif reading_source['type'] == "psalm-chanted":
        filename_add = "-chanted"
    elif reading_source['type'] == "prayer_of_day":
        filename_add = ""
    else:
        filename_add = ""

    filename_out = "./out_" + reading_source['title'] + filename_add + ".pro"
    print("Writing output file: "+filename_out)
    with open(filename_out, "wb") as fd:
        fd.write(proto_pres_obj_mod.SerializeToString())
    #return proto_pres_obj_mod


In [43]:
#BEGIN PROGRAM EXECUTION

worship_date = "2023-5-21"

In [44]:
#Set up and do Web login stuff
print("Setting up Web driver...")
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

#Navigate to login page and log in
print("Logging into Sundays and Seasons website...")
login_url = "https://members.sundaysandseasons.com/Account/Login?returnUrl=%2F"
driver.get(login_url)
page_source = driver.page_source
username = "kidspace"
password = "sudbury01776"
driver.find_element(By.XPATH,'//*[@id="UserName"]').send_keys(username)
driver.find_element(By.XPATH,'//*[@id="Password"]').send_keys(password)
driver.find_element(By.XPATH,'//*[@id="loginForm"]/form/fieldset/input').click()
url = driver.current_url
#print(url)

#Navigate to daily texts page
print("Navigating to texts for " + worship_date + "...")

daily_texts_url = "https://members.sundaysandseasons.com/Home/TextsAndResources/"+worship_date+"/0#texts"

#daily_texts_url = "https://members.sundaysandseasons.com/Home/TextsAndResources#texts"
driver.get(daily_texts_url)
url = driver.current_url
print(url)

#Click texts button and wait/load page
print("Extracting and cleaning up daily texts...")
text_container = driver.find_element(By.ID,'toggle-btn-panel-texts')
driver.execute_script("arguments[0].style.display = 'block';", text_container)
time.sleep(4)
page_source = driver.page_source
#print("Now")

#Hand page to Beautiful Soup for processing
##page = BeautifulSoup(page_source, 'lxml')
soup = BeautifulSoup(page_source, "lxml",
                     parse_only = SoupStrainer(
                       'div', id = 'toggle-btn-panel-texts'))

#Remove rubric divs
print("\t"+"Removing extra div tags...")
for div in soup.find_all('div', {'class':'rubric'}): 
    div.decompose()

#Remove reading intros
print("\t"+"Removing reading intros...")
for div in soup.find_all('div', {'class':'reading_intro'}):
    div.decompose()

#Remove verse markers
print("\t"+"Removing verse markers...")
for sup_div in soup.find_all("span",style = "white-space: nowrap;"):
    sup_div.decompose()

#Process readings
print("Loading in reading texts...")
print("\t"+"First Reading:",end=" ")
first_reading_header = first_reading = soup.find('h3',string=re.compile('First Reading:')).getText()
first_reading_source = get_source_info(first_reading_header)
first_reading = soup.find('h3',string=re.compile('First Reading:')).findNext('div').getText()
first_reading = " ".join(line.strip() for line in first_reading.split("\n"))
print(first_reading_source['type'], first_reading_source['title'],first_reading_source['book'],first_reading_source['number'])
print("\t\t",first_reading)

print("\t"+"Second Reading:",end=" ")
second_reading_header = soup.find('h3',string=re.compile('Second Reading:')).getText()
second_reading_source = get_source_info(second_reading_header)
second_reading = soup.find('h3',string=re.compile('Second Reading:')).findNext('div').getText()
second_reading = " ".join(line.strip() for line in second_reading.split("\n"))
print(second_reading_source['type'], second_reading_source['title'],second_reading_source['book'],second_reading_source['number'])
print("\t\t",second_reading)

print("\t"+"Gospel:",end=" ")
gospel_header = soup.find('h3',string=re.compile('Gospel:')).getText()
gospel_source = get_source_info(gospel_header)
gospel = soup.find('h3',string=re.compile('Gospel:')).findNext('div').getText()
gospel = " ".join(line.strip() for line in gospel.split("\n"))
print(gospel_source['type'], gospel_source['title'],gospel_source['book'],gospel_source['number'])
print("\t\t",gospel,end="\n\n")


#Load and process psalm
print("Loading in Psalm text...")
psalm_slides_chanted = []
psalm_slides_spoken = []

psalm_html = soup.find('h3',string=re.compile('Psalm:')).findNext('div')

psalm_header = soup.find('h3',string=re.compile('Psalm:')).getText()

psalm_source_chanted = get_source_info(psalm_header)
psalm_source_chanted['type'] = 'psalm-chanted'
psalm_source_spoken = get_source_info(psalm_header)
psalm_source_spoken['type'] = 'psalm-spoken'

for span in psalm_html.find_all('span', {'style':'white-space: nowrap;'}):
    span.decompose()
for span in psalm_html.find_all('span', {'style':'font-variant: small-caps;'}):
    span.unwrap()
for span in psalm_html.find_all('span', {'class':'refrain'}):
    span.decompose()

print("Processing tabs and separator marks for chanted psalm...")
for span in psalm_html.find_all('span'):
    line = span.text.strip()
    if line != "":
        psalm_slides_spoken.append(line.split("|")[0].strip() + " " + line.split("|")[1].strip())
        psalm_slides_chanted.append(line.split("|")[0].strip() + "\t|\t" + line.split("|")[1].strip())

for i in range(len(psalm_slides_spoken)):
    if "- " in psalm_slides_spoken[i]:
        psalm_slides_spoken[i] = psalm_slides_spoken[i].replace("- ","")


print("\tChanted: ", psalm_source_chanted)
i=0
for slide in psalm_slides_chanted[0:2]:
    print("\t\t",i,slide)
    i+=1
print("\t\t...")

#print("\n")
i=0
print("\tSpoken: ",psalm_source_spoken)
for slide in psalm_slides_spoken[0:2]:
    print("\t\t",i,slide)
    i+=1
print("\t\t...",end="\n\n")

#Compile other readings
print("Loading in other worship elements...",end="\n")

print("\tLoading Prayers of Intercession...",end="\n")
prayers_intercession = soup.find('h3',string=re.compile('Prayers of Intercession')).findNext('div').getText()

print("\tLoading and processing Prayer of the Day...")
prayer_of_day_text_raw = soup.find('h3',string=re.compile('Prayer of the Day')).findNext('div').getText()
prayer_of_day_header = "Prayer of the Day Prayer:"+prayer_of_day_text_raw.split(",")[0].strip()
prayer_of_day_textlist = prayer_of_day_text_raw.split(",")[1:]
prayer_of_day = ",".join(prayer_of_day_textlist).split("Amen")[0].strip()
prayer_of_day_source = get_source_info(prayer_of_day_header)
print("\t\t"+prayer_of_day,end="\n")

#print("\tLoading Communion Prayer...")
#post_communion_prayer = soup.find('h3',string=re.compile('Prayer after Communion')).findNext('div').getText().strip()
#post_communion_prayer = " ".join(line.strip() for line in post_communion_prayer.split("\n"))
#print("\t\t"+post_communion_prayer,end="\n")

#print("\tLoading Blessing...")
#blessing = soup.find('h3',string=re.compile('Blessing')).findNext('div').getText().strip()
#blessing = " ".join(line.strip() for line in blessing.split("\n"))
#print("\t\t"+blessing,end="\n")

#print("\tLoading Dismissal...")
#dismissal = soup.find('h3',string=re.compile('Dismissal')).findNext('div').getText().strip()
#dismissal = " ".join(line.strip() for line in dismissal.split("\n"))
#print("\t\t"+dismissal,end="\n\n")

#Parse readings into sentences and distribute into slides
print("Parsing readings into sentences and distributing into slides...",end="\n")
first_reading_tokenized = tokenize_reading(first_reading)
first_reading_slides = slides_from_reading(first_reading_tokenized,300)

second_reading_tokenized = tokenize_reading(second_reading)
second_reading_slides = slides_from_reading(second_reading_tokenized,300)

gospel_tokenized = tokenize_reading(gospel)
gospel_slides = slides_from_reading(gospel_tokenized,300)

prayer_of_day_tokenized = tokenize_reading(prayer_of_day)
prayer_of_day_slides = slides_from_reading(prayer_of_day_tokenized,300)

Setting up Web driver...


[WDM] - Downloading: 100%|██████████| 8.06M/8.06M [00:01<00:00, 7.44MB/s]


Logging into Sundays and Seasons website...
Navigating to texts for 2023-5-21...
https://members.sundaysandseasons.com/Home/TextsAndResources/2023-5-21/0#texts
Extracting and cleaning up daily texts...
	Removing extra div tags...
	Removing reading intros...
	Removing verse markers...
Loading in reading texts...
	First Reading: reading First-Reading Acts 1
		 When [the apostles] had come together, they asked [Jesus], “Lord, is this the time when you will restore the kingdom to Israel?” He replied, “It is not for you to know the times or periods that the Father has set by his own authority. But you will receive power when the Holy Spirit has come upon you; and you will be my witnesses in Jerusalem, in all Judea and Samaria, and to the ends of the earth.” When he had said this, as they were watching, he was lifted up, and a cloud took him out of their sight. While he was going and they were gazing up toward heaven, suddenly two men in white robes stood by them. They said, “Men of Galilee,

In [45]:
#Transform text into screen overlay titles and export to slide files
print("Transforming text into screen overlay titles and exporting to slide files...",end="\n")
make_reading_presentation(first_reading_slides,first_reading_source)
make_reading_presentation(second_reading_slides,second_reading_source)
make_reading_presentation(gospel_slides,gospel_source)
make_reading_presentation(prayer_of_day_slides,prayer_of_day_source)
make_reading_presentation(psalm_slides_chanted,psalm_source_chanted)
make_reading_presentation(psalm_slides_spoken,psalm_source_spoken)

Transforming text into screen overlay titles and exporting to slide files...
Template loaded: Reading_template_slides.pro. Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...
Writing output file: ./out_First-Reading.pro
Template loaded: Reading_template_slides.pro. Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...
Writing output file: ./out_Second-Reading.pro
Template loaded: Reading_template_slides.pro. Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...
Writing output file: ./out_Gospel.pro
Template loaded: Prayer_of_day_template.pro. Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...
Writing output file: ./out_Prayer-of-the-day.pro
Template loaded: Psalm_template_chanted.pro. Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...
Writing output file: ./out_Psalm

In [24]:
make_reading_presentation(psalm_slides_chanted,psalm_source_chanted)

Template loaded: Psalm_template_chanted.pro. Template file confirmed as ProPresenter 7 Presentation file. Loading as protocol buffer class...
Writing output file: ./out_Psalm-chanted.pro


In [14]:
i=0
for slide in psalm_slides_chanted:
    print(i, slide)
    i+=1

0 I love the Lord, who has	|	heard my voice,
1 and listened to my	|	supplication,
2 for the Lord has given	|	ear to me
3 whenev-	|	er I called.
4 How shall I re-	|	pay the Lord
5 for all the good things God has	|	done for me?
6 I will lift the cup	|	of salvation
7 and call on the name	|	of the Lord.
8 I will fulfill my vows	|	to the Lord
9 in the presence of	|	all God’s people.
10 Precious in your	|	sight, O Lord,
11 is the death	|	of your servants.
12 O Lord, truly I	|	am your servant;
13 I am your servant, the child of your handmaid; you have freed me	|	from my bonds.
14 I will offer you the sacrifice	|	of thanksgiving
15 and call upon the name	|	of the Lord.
16 I will fulfill my vows	|	to the Lord
17 in the presence of	|	all God’s people,
18 in the courts of	|	the Lord’s house,
19 in the midst of you,	|	O Jerusalem.
